In [ ]:
import os
from glob import glob

from datasets import load_dataset
import pandas as pd

from paths.paths_2013 import TEXTS_DIR, RESULTS_DIR, DATA_DIR, YEAR

In [ ]:
def write_to_file(df_chunk):
    results_dir = TEXTS_DIR
    
    df = pd.DataFrame(df_chunk)
    grouped = df.groupby('subreddit')['body'].apply(lambda x: ' '.join(x)).reset_index()
    for idx, row in grouped.iterrows():
        output_file = 'subreddit_' + row['subreddit'] + '.txt'
        with open(os.path.join(results_dir, output_file), 'a') as f:
            f.write(row['body'])
    return df_chunk

In [ ]:
def comments_by_subreddit():
    data_dir = DATA_DIR
    
    data = load_dataset('parquet', data_files=os.path.join(data_dir, f'RC_{YEAR}*'), split='train', streaming=True)
    data_mapped = data.map(write_to_file, batched=True, batch_size=10000)
    for data in data_mapped:
        pass

In [ ]:
def texts_to_single_file():
    data_dir = TEXTS_DIR
    
    files = glob(os.path.join(data_dir, 'subreddit_*'))
    files.sort()
    for file in files:
        append_file(file)

In [ ]:
def append_file(file):
    results_dir = RESULTS_DIR
    
    with open(file, 'r') as input_file:
        with open(os.path.join(results_dir, 'subreddits.txt'), 'a') as output_file:
            for text in input_file.read(int(1e8)):
                output_file.write(text)
            output_file.write('\n')

In [ ]:
comments_by_subreddit()

In [ ]:
texts_to_single_file()